<a href="https://colab.research.google.com/github/MykhailoFokin/MachineLearning/blob/Dev/Face_landmarks_ML2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Copy images repo
!git clone https://github.com/SilvesterHsu/ORLFaceRecognition-PCA.git

#Copy shape_predictor_68_face_landmarks.dat file
import urllib.request
urllib.request.urlretrieve('http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2', 'shape_predictor_68_face_landmarks.dat.bz2')

#Uncompress file
import bz2,shutil

with bz2.BZ2File("shape_predictor_68_face_landmarks.dat.bz2") as fr, open("shape_predictor_68_face_landmarks.dat","wb") as fw:
    shutil.copyfileobj(fr,fw)


Cloning into 'ORLFaceRecognition-PCA'...
remote: Enumerating objects: 604, done.
remote: Total 604 (delta 0), reused 0 (delta 0), pack-reused 604
Receiving objects: 100% (604/604), 3.59 MiB | 5.22 MiB/s, done.
Resolving deltas: 100% (93/93), done.


In [0]:
#Initialize simple XML fuction
def dict2xml(d, root_node=None):
	wrap          =     False if None == root_node or isinstance(d, list) else True
	root          = 'objects' if None == root_node else root_node
	root_singular = root[:-1] if 's' == root[-1] and None == root_node else root
	xml           = ''
	children      = []

	if isinstance(d, dict):
		for key, value in dict.items(d):
			if isinstance(value, dict):
				children.append(dict2xml(value, key))
			elif isinstance(value, list):
				children.append(dict2xml(value, key))
			else:
				xml = xml + ' ' + key + '="' + str(value) + '"'
	else:
		for value in d:
			children.append(dict2xml(value, root_singular))

	end_tag = '>' if 0 < len(children) else '/>'

	if wrap or isinstance(d, dict):
		xml = '<' + root + xml + end_tag

	if 0 < len(children):
		for child in children:
			xml = xml + child

		if wrap or isinstance(d, dict):
			xml = xml + '</' + root + '>'
		
	return xml

In [0]:
# ORL database
# https://github.com/SilvesterHsu/ORLFaceRecognition-PCA/tree/origin/master/att_faces

# dlib face landmark detection example
# http://dlib.net/face_landmark_detection.py.html

# 68 face landmarks annotations
# https://ibug.doc.ic.ac.uk/resources/facial-point-annotations/

# Example of result
"""
[
  {
    "img": "s1/1.pgm"
    "left_eye_x": 20,
    "left_eye_y": 52,
    "right_eye_x": 69,
    "right_eye_y": 51
  }
]
"""
import os
import fnmatch
import json
import cv2
import dlib
import matplotlib.pyplot as plt
from google.colab import drive
from google.colab import files as gfile
%matplotlib inline

# drive.mount('/content/drive', force_remount=True)

images_matrix = []

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

# read folder structure with files and put it to collection
folder = []
for i in os.walk(os.path.join('ORLFaceRecognition-PCA','att_faces')):
  folder.append(i)

for address, dirs, files in folder:
    
    for filename in fnmatch.filter(files, '*.pgm'):

      img = cv2.imread(os.path.join(address,filename))
      gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
      
      faces, scores, idx = detector.run(gray, 1, -0.6)

      #faces = detector(gray,1)
      for face in faces:
        landmarks = predictor(gray, face)

        x1 = landmarks.part(36).x
        y1 = landmarks.part(36).y

        x2 = landmarks.part(45).x
        y2 = landmarks.part(45).y

        #cv2.circle(img, (x1, y1), 2, (255, 0, 0), -1)
        #cv2.circle(img, (x2, y2), 2, (255, 0, 0), -1)
        #plt.imshow(img)

        images_matrix.append({"img": os.path.join(address.replace('ORLFaceRecognition-PCA/att_faces/',''),filename),
                              "left_eye_x": x1,
                              "left_eye_y": y1,
                              "right_eye_x": x2,
                              "right_eye_y": y2
                            })


images_matrix.sort(key=lambda k : k['img'])
#print(images_matrix)

# Save as json into colab
#with open("data_file.json", "w") as write_file:
#    json.dump(images_matrix, write_file)

# Save as json into google drive
#with open('/content/drive/My Drive/Colab Notebooks/file.txt', 'w') as f:
#  json.dump(images_matrix, f)

# Download file from colab to local computer
#gfile.download("data_file.json")

# Save as XML to colab
import re 
xml_title = "Result"
xml_tag_pattern = re.compile(r'</?{}>'.format(xml_title))
inner_xml = re.sub(xml_tag_pattern, '', ''.join(dict2xml(e, root_node='Image') for e in images_matrix))

with open("data_file.xml", "w") as write_file:
    write_file.write('<{0}>{1}</{0}>'.format(xml_title, inner_xml))